In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
import os
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme()

In [3]:
data_dir = "/data/ddmg/redditlanguagemodeling/data/AmazonReviews/data"

In [4]:
data_df = pd.read_csv(os.path.join(data_dir, 'amazon_v2.0/reviews.csv'),
                      dtype={'reviewerID':str, 'asin':str, 'reviewTime':str,'unixReviewTime':int,
                             'reviewText':str,'summary':str,'verified':bool,'category':str, 'reviewYear':int},
                      keep_default_na=False, na_values=[], quoting=csv.QUOTE_NONNUMERIC)

In [5]:
split_df = pd.read_csv(os.path.join(data_dir, 'amazon_v2.0', 'splits', 'user.csv'))

In [6]:
used_df = data_df[split_df["split"] != -1]

In [14]:
# create train, test, val split --> 60-20-20
train_count = int(len(used_df) * .6)
val_count = int(len(used_df) * .2)
test_count = len(used_df) - train_count - val_count
print("train count", train_count)
print("val count", val_count)
print("test count", test_count)
split_assignments = np.concatenate([np.zeros(train_count), np.ones(val_count), np.full(test_count, 2)])
np.random.shuffle(split_assignments)

train count 323701
val count 107900
test count 107901


In [20]:
# create new split file
split_df.loc[used_df.index, "split"] = split_assignments

In [23]:
np.unique(split_df["split"], return_counts=True)

(array([-1.,  0.,  1.,  2.]), array([9577445,  323701,  107900,  107901]))

In [24]:
split_df.to_csv(os.path.join(data_dir, 'amazon_v2.0', 'splits', 'mlm.csv'), index=False)